In [108]:
%load_ext autoreload
%autoreload 2
import os
import unittest

import pandas as pd
from pathlib import Path
import pickle
import numpy as np
import warnings

warnings.filterwarnings("ignore")

from kmdcm.pydcm.dcm import (
    mdcm_set_up,
    eval_kernel,
)
from kmdcm.pydcm.dcm import FFE_PATH, espform, densform
from kmdcm.utils import dcm_utils as du
from kmdcm.pydcm.mdcm_dict import MDCM
from kmdcm.pydcm.kernel import KernelFit
from kmdcm.pydcm.dcm import *
# Optimization
from scipy.optimize import minimize


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
# cubes = list(Path("/pchem-data/meuwly/boittier/home/ref/comformation-cube/Benz").glob("esp*.cube"))
# str_cubes = [str(_) for _ in cubes if "mdcm" not in str(_)]
# str_cubes.sort()
# str_cubes = str_cubes[::-1]
# str_cubes

In [110]:
cubes = list(Path("/pchem-data/meuwly/boittier/home/dcm").glob("esp*.cube"))
str_cubes = [str(_) for _ in cubes if "mdcm" not in str(_)]
str_cubes.sort()
str_cubes = str_cubes[::-1]
str_cubes

['/pchem-data/meuwly/boittier/home/dcm/esp-dcm4.cube',
 '/pchem-data/meuwly/boittier/home/dcm/esp-dcm3.cube',
 '/pchem-data/meuwly/boittier/home/dcm/esp-dcm2.cube',
 '/pchem-data/meuwly/boittier/home/dcm/esp-dcm1.cube',
 '/pchem-data/meuwly/boittier/home/dcm/esp-dcm.cube']

In [111]:
Nfiles = len(str_cubes)
model_dir = Path("/pchem-data/meuwly/boittier/home/mdcm_fast/mdcm/dcm/7-8")
mdcm_xyz = model_dir / "dc.xyz"
mdcm_model = model_dir / "model.mdcm"
mdcm = mdcm_set_up(str_cubes, str_cubes, mdcm_cxyz=mdcm_xyz, mdcm_clcl=mdcm_model, local_pos=None)

test:
/pchem-data/meuwly/boittier/home/dcm/esp-dcm4.cube
Reading '/pchem-data/meuwly/boittier/home/dcm/esp-dcm4.cube'... just atom number 
/pchem-data/meuwly/boittier/home/mdcm_fast/mdcm/dcm/7-8/dc.xyz
cxyz: [-2.88266382e-05  1.33954841e+00 -1.70075351e-09 -7.13125527e-01
 -2.88993927e-05  1.31320466e+00 -1.32280829e-09 -1.03970366e-02
 -6.63161589e-06  2.65547222e+00  1.91661898e+00  2.56479859e-01
 -7.74768814e-06  2.65547222e+00 -1.91661898e+00  2.56479829e-01
 -2.93508343e+00 -5.14684676e-01  1.19052746e-08  1.13654859e-01
 -2.84031157e+00 -4.23216581e-01 -6.23609621e-09 -8.37111290e-03
  2.93509154e+00 -5.14675327e-01  1.19052746e-08  1.13651574e-01
  2.84032036e+00 -4.23206782e-01 -1.03934937e-08 -8.37246790e-03]
/pchem-data/meuwly/boittier/home/mdcm_fast/mdcm/dcm/7-8/model.mdcm
[-6.38102706e-03  1.19052746e-08  1.80575375e-01  1.13654859e-01
  1.71169677e-02 -6.23609621e-09  5.09761378e-02 -8.37111290e-03
 -9.26612659e-02 -1.70075351e-09  6.18169208e-02 -7.13125527e-01
 -1.14576

In [112]:
# Get RMSE, averaged or weighted over ESP files, or per ESP file each
rmse = mdcm.get_rmse()
print(rmse)
srmse_start = mdcm.get_rmse_each(Nfiles)
print(srmse_start)
srmse_start[0] = 1
srmse_start[1:] = 1
wrmse = mdcm.get_rmse_weighted(Nfiles, srmse_start)
print(wrmse)


4.127213512975496
[4.13729922 4.12243288 4.1285055  4.12120008 4.12660966]
4.127213512975496


In [113]:
clcl = mdcm.mdcm_clcl
xyzq = clcl.reshape(clcl.shape[0]//4,4)
q = xyzq[:,-1]
q = np.round(q, 10)
unique_q = list(set(q))
unique_q, indices = np.unique(q, return_inverse=True)
print("Unique charges: ", unique_q)
clcl = mdcm.mdcm_clcl
rmse = mdcm.get_rmse()
l2 = 0.1
#  save an array containing original charges
charges = clcl.copy().reshape(clcl.shape[0]//4,4)
local_pos = clcl[np.mod(np.arange(clcl.size) + 1, 4) != 0]
local_ref = None #local_pos.copy()


Unique charges:  [-0.71312553 -0.01039704 -0.00837247 -0.00837111  0.11365157  0.11365486
  0.25647983  0.25647986]


In [114]:
indices

array([5, 3, 0, 1, 4, 2, 7, 6])

In [115]:

clcl = mdcm.mdcm_clcl
xyzq = clcl.reshape(clcl.shape[0]//4,4)
lclc = xyzq[:,:-1]
lclc = np.round(lclc.flatten(), 10)
lclc_sgn = np.sign(lclc)
unique_lclc, lclc_indices = np.unique(abs(lclc), return_inverse=True)
print("Unique local positions", unique_lclc)

def mdcm_rmse(unique_lclc, local_ref=local_ref, srmse_start=srmse_start, charges=charges, l2=l2):
    """Minimization routine"""
    lp = np.take(unique_lclc, lclc_indices)*lclc_sgn
    _clcl_ = get_clcl(lp, charges.flatten())
    mdcm.set_clcl(_clcl_)
    rmse = mdcm.get_rmse_weighted(Nfiles, srmse_start)
    # print(rmse)
    if local_ref is not None:
        l2diff = l2 * np.sum((local_pos - local_ref) ** 2) / local_pos.shape[0]
        rmse += l2diff
    return rmse

def mdcm_q_rmse(q, local_ref=local_ref, srmse_start=srmse_start, charges=charges, l2=l2):
    """Minimization routine"""
    # _clcl_ = get_clcl(q, charges)
    # print(q)
    q = np.take(q, indices)
    charges[:,-1] = q
    mdcm.set_clcl(charges.flatten())
    rmse = mdcm.get_rmse_weighted(Nfiles, srmse_start)
    # print(rmse)
    if local_ref is not None:
        l2diff = l2 * np.sum((local_pos - local_ref) ** 2) / local_pos.shape[0]
        rmse += l2diff
    return rmse



Unique local positions [1.30000000e-09 1.70000000e-09 6.20000000e-09 1.04000000e-08
 1.19000000e-08 6.38102710e-03 6.38228920e-03 1.71169677e-02
 1.71180108e-02 3.39399342e-02 3.39399470e-02 5.09737794e-02
 5.09761378e-02 6.13099276e-02 6.13099346e-02 6.18169208e-02
 7.64363073e-02 9.26612659e-02 1.14576256e-01 1.80572423e-01
 1.80575375e-01 1.98269102e-01]


In [116]:
lp = np.take(unique_lclc, lclc_indices)*lclc_sgn
lp.reshape(len(lp)//3, 3)

array([[-6.38102710e-03,  1.19000000e-08,  1.80575375e-01],
       [ 1.71169677e-02, -6.20000000e-09,  5.09761378e-02],
       [-9.26612659e-02, -1.70000000e-09,  6.18169208e-02],
       [-1.14576256e-01, -1.30000000e-09,  7.64363073e-02],
       [ 6.38228920e-03,  1.19000000e-08,  1.80572423e-01],
       [-1.71180108e-02, -1.04000000e-08,  5.09737794e-02],
       [-3.39399342e-02,  6.13099346e-02,  1.98269102e-01],
       [-3.39399470e-02, -6.13099276e-02,  1.98269102e-01]])

In [117]:
for i in range(1):
    print("L-BFGS-B run:", i+1)
    # Apply simple minimization without any feasibility check (!)
    # Leads to high amplitudes of MDCM charges and local positions
    res = minimize(
        # mdcm_rmse,
        mdcm_q_rmse,
        unique_q,
        method="L-BFGS-B",
        # method="Nelder-Mead",
        bounds=[(-1, 1)] * len(unique_q),
        options={
            "disp": None,
            "maxls": 20,
            "adaptive": True,
            "iprint": -1,
            "gtol": 1e-7,
            "eps": 1e-4,
            "maxiter": 1000,
            "ftol": 1e-7,
            "factr": 1e1,
            "maxcor": 10,
            "maxfun": 15000,
        },
    )
    print(res)
    print(res.x)
    
    # Get RMSE, averaged or weighted over ESP files, or per ESP file each
    rmse = mdcm.get_rmse()
    print(rmse)
    wrmse = mdcm.get_rmse_weighted(Nfiles, 1/srmse_start)
    print(wrmse)
    srmse = mdcm.get_rmse_each(Nfiles)
    print(srmse)


L-BFGS-B run: 1
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1.4492758465085462
        x: [-4.559e-01  2.915e-01 -1.000e+00 -9.959e-01  8.942e-01
             8.902e-01  1.895e-01  1.896e-01]
      nit: 43
      jac: [ 1.892e-01  1.741e-01  3.322e-01  1.971e-01  2.334e-01
             9.357e-02  1.959e-01  1.945e-01]
     nfev: 522
     njev: 58
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
[-0.45594382  0.29154676 -1.         -0.99594671  0.89417041  0.89016104
  0.18954011  0.18958083]
1.4492952963561878
1.4492952963561878
[1.4495767  1.44896714 1.4497664  1.44900986 1.44915614]


In [106]:
# Get RMSE, averaged or weighted over ESP files, or per ESP file each
rmse = mdcm.get_rmse()
print(rmse)
srmse_start = mdcm.get_rmse_each(Nfiles)
print(srmse_start)
srmse_start = 2 * srmse_start/srmse_start[0]
srmse_start[0] = 1
print(srmse_start)
wrmse = mdcm.get_rmse_weighted(Nfiles, srmse_start)
print(wrmse)

clcl = mdcm.mdcm_clcl
xyzq = clcl.reshape(clcl.shape[0]//4,4)
q = xyzq[:,-1]
q = np.round(q, 3)
unique_q = list(set(q))
unique_q, indices = np.unique(q, return_inverse=True)
print(unique_q)
clcl = mdcm.mdcm_clcl
rmse = mdcm.get_rmse()
l2 = 0.1
print(rmse)
print("clcl: ", clcl.shape)
#  save an array containing original charges
charges = clcl.copy().reshape(clcl.shape[0]//4,4)
print(charges)
local_pos = clcl[np.mod(np.arange(clcl.size) + 1, 4) != 0]
local_ref = None #local_pos.copy()

clcl = mdcm.mdcm_clcl
xyzq = clcl.reshape(clcl.shape[0]//4,4)
lclc = xyzq[:,:-1]
lclc = np.round(lclc.flatten(), 10)
lclc_sgn = np.sign(lclc)
unique_lclc, lclc_indices = np.unique(abs(lclc), return_inverse=True)
print("Unique local positions", unique_lclc)

def mdcm_rmse(unique_lclc, local_ref=local_ref, srmse_start=srmse_start, charges=charges, l2=l2):
    """Minimization routine"""
    lp = np.take(unique_lclc, lclc_indices)*lclc_sgn
    _clcl_ = get_clcl(lp, charges.flatten())
    mdcm.set_clcl(_clcl_)
    rmse = mdcm.get_rmse_weighted(Nfiles, srmse_start*0+1)
    # print(rmse)
    if local_ref is not None:
        l2diff = l2 * np.sum((local_pos - local_ref) ** 2) / local_pos.shape[0]
        rmse += l2diff
    return rmse

def mdcm_q_rmse(q, local_ref=local_ref, srmse_start=srmse_start, charges=charges, l2=l2):
    """Minimization routine"""
    # _clcl_ = get_clcl(q, charges)
    # print(q)
    q = np.take(q, indices)
    charges[:,-1] = q
    mdcm.set_clcl(charges.flatten())
    rmse = mdcm.get_rmse_weighted(Nfiles, srmse_start*0+1)
    # print(rmse)
    if local_ref is not None:
        l2diff = l2 * np.sum((local_pos - local_ref) ** 2) / local_pos.shape[0]
        rmse += l2diff
    return rmse



# # Apply simple minimization without any feasibility check (!)
# # Leads to high amplitudes of MDCM charges and local positions
res = minimize(
    # mdcm_rmse,
    mdcm_q_rmse,
    unique_q,
    method="L-BFGS-B",
    # method="Nelder-Mead",
    bounds=[(-1, 1)] * len(unique_q),
    options={
        "disp": None,
        "maxls": 20,
        "adaptive": True,
        "iprint": 1,
        "gtol": 1e-4,
        "eps": 1e-7,
        "maxiter": 300,
        "ftol": 1e-7,
        "factr": 1e7,
        "maxcor": 10,
        "maxfun": 15000,
    },
)
print(res)
print(res.x)

# Get RMSE, averaged or weighted over ESP files, or per ESP file each
rmse = mdcm.get_rmse()
print(rmse)
wrmse = mdcm.get_rmse_weighted(Nfiles, 0*srmse_start+1)
print(wrmse)
srmse = mdcm.get_rmse_each(Nfiles)
print(srmse)

clcl = mdcm.mdcm_clcl
rmse = mdcm.get_rmse()
l2 = 0.1
print(rmse)
print("clcl: ", clcl.shape)
#  save an array containing original charges
charges = clcl.copy()

# Apply simple minimization without any feasibility check (!)
# Leads to high amplitudes of MDCM charges and local positions
res = minimize(
    mdcm_rmse,
    unique_lclc,
    method="L-BFGS-B",
    # method="Nelder-Mead",
    bounds=[(-0.55, 0.55)] * len(unique_lclc),
    options={
        "disp": None,
        "maxls": 20,
        "adaptive": True,
        "iprint": 1,
        "gtol": 1e-4,
        "eps": 1e-7,
        "maxiter": 300,
        "ftol": 1e-7,
        "factr": 1e7,
        "maxcor": 10,
        "maxfun": 15000,
    },
)
print(res)
print(res.x)




1.0789707285749908
[1.07884475 1.07835147 1.08009076 1.07882686 1.07873886]
[1.         1.99908554 2.00230989 1.99996683 1.9998037 ]
0.835711166345039
[-1.    -0.417  0.183  0.29   0.882  0.883]
1.0789707285749908
clcl:  (32,)
[[ 4.17612195e-01  8.73712351e-04  5.50000000e-01  8.82095580e-01]
 [ 4.99931112e-01  8.33504237e-04  2.99714928e-01 -1.00000000e+00]
 [-5.50000000e-01 -2.97989364e-04  4.78591006e-01 -4.16906699e-01]
 [ 3.85998399e-01  1.19107357e-03 -1.57630372e-01  2.89556085e-01]
 [-4.24678524e-01  8.73712351e-04  5.50000000e-01  8.83212566e-01]
 [-5.03846327e-01  1.20427655e-03  3.14556353e-01 -1.00000000e+00]
 [-3.29482718e-01  5.47906014e-01 -4.51923333e-02  1.82537860e-01]
 [-3.29385544e-01 -5.46983095e-01 -4.51923333e-02  1.82537860e-01]]
Unique local positions [2.97989400e-04 8.33504200e-04 8.73712400e-04 1.19107360e-03
 1.20427660e-03 4.51923333e-02 1.57630372e-01 2.99714928e-01
 3.14556353e-01 3.29385544e-01 3.29482718e-01 3.85998399e-01
 4.17612195e-01 4.24678524e-01

In [107]:
# Get RMSE, averaged or weighted over ESP files, or per ESP file each
rmse = mdcm.get_rmse()
print(rmse)
srmse = mdcm.get_rmse_each(Nfiles)
print(srmse)

1.0689568440021397
[1.06893948 1.0683433  1.06998995 1.06878432 1.06872633]


In [101]:
mdcm.write_cxyz_files()
mdcm.write_mdcm_cube_files()

test:
/pchem-data/meuwly/boittier/home/dcm/esp-dcm4.cube                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                